In [ ]:
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        'google-cloud-bigquery[pandas]'

In [ ]:
import IPython

# Restart kernel
app = IPython.Application.instance()
app.kernel.do_shutdown(True)


In [3]:
# Set env vars
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-east4"

## Initialise Vertex AI Python SDK

In [4]:
import vertexai
vertexai.init(project = PROJECT_ID,
              location = REGION)

## Prepare Data in BigQuery

The [StackOverflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow) is a BQ mirror of Stack Overflow public dataset.

In [5]:
import math
from typing import Any, Generator

import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

In [7]:
# query to retrieve data from public Stack Overflow dataset
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title, q.body
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q
        LIMIT {limit} OFFSET {offset};
        """

In [9]:
# Access the data in chunks to make it manageable in memory
def query_bigquery_chunks(max_rows: int, rows_per_chunk: int, start_chunk: int = 0) -> Generator[pd.DataFrame, Any, None]:
    for offset in range(start_chunk, max_rows, rows_per_chunk):
        query = QUERY_TEMPLATE.format(limit=rows_per_chunk, offset=offset)
        query_job = client.query(query)
        rows = query_job.result()
        df = rows.to_dataframe()
        df["title_with_body"] = df.title + "\n" + df.body
        yield df

In [10]:
# Get a dataframe of 1000 riows for demo purposes
df = next(query_bigquery_chunks(max_rows=1000, rows_per_chunk=1000))

# Examine the data
df.head()

,id,title,body,title_with_body
0,13052878,emacs 24 c++ auto-indentation broken,"<p>I'm running Emacs 24 on Ubuntu 10.04, codin...",emacs 24 c++ auto-indentation broken\n<p>I'm r...
1,12727898,UIKeyboard avoidance and Auto Layout,"<p>Given the focus on Auto Layout in iOS 6, an...",UIKeyboard avoidance and Auto Layout\n<p>Given...
2,12848712,Changed project name in Xcode causing naming e...,<p>My old name consisted of a camel case type ...,Changed project name in Xcode causing naming e...
3,12829700,Android Unit Testing: Bundle/Parcelable,<p>How do you unit test Parcelable? I created...,Android Unit Testing: Bundle/Parcelable\n<p>Ho...
4,12680080,python enums with attributes,<p>Consider:</p>\n<pre><code>class Item:\n d...,python enums with attributes\n<p>Consider:</p>...


## Create Text Embeddings from BigQuery Data

In [11]:
from typing import List, Optional
from vertexai.preview.language_models import TextEmbeddingModel

# Load the Vertex AI "Embeddings for Text" model
model = TextEmbeddingModel.from_pretrained("text-embedding-004")

In [12]:
# Define an embedding function that uses the model
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

In [13]:
import functools
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Generator, List, Tuple

import numpy as np
from tqdm.auto import tqdm

# Generator function to yield batches of sentences
def generate_batches(
    sentences: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

In [14]:
def encode_text_to_embedding_batched(
    sentences: List[str], api_calls_per_second: int = 10, batch_size: int = 5
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total=math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job) # for rate limiting - in Prod, we would do something more sophisticated

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return is_successful, embeddings_list_successful

In [15]:
# Encode a subset of questions for validation
questions = df.title.tolist()[:500]
is_successful, question_embeddings = encode_text_to_embedding_batched(
    sentences=df.title.tolist()[:500]
)

# Filter for successfully embedded sentences
questions = np.array(questions)[is_successful]

  0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
# Save the dimension size for later usage when creating the Vertex AI Vector Search index.
DIMENSIONS = len(question_embeddings[0])

print(DIMENSIONS)

768


## Sort questions by order of similarity

- 0 means very different
- 1 means very similar

In [18]:
import random

question_index = random.randint(0, 99)

print(f"Query question = {questions[question_index]}")

# Get similarity scores for each embedding by using dot-product.
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

Query question = Setting "an informative User-Agent string" in getURL
	0: Setting "an informative User-Agent string" in getURL: 0.9999981819202963
	1: Getting JSON data: 0.5050393007723325
	2: Is it possible to have open-uri maintain the extension?: 0.4645424495835494
	3: Is it possible to return HTTP response code to the browser using pl/sql?: 0.4431916192040807
	4: UploadStringAsync CallBack Return Type: 0.4378561185815355
	5: handle jsonp with varnish: 0.4326099188977214
	6: getting name value pairs(in decoded format) from the posted form: 0.42829365460803087
	7: 301 redirect vs canonical links?: 0.4216442393417451
	8: Open remote file and write to it: 0.4213806729405419
	9: How to to set a background image on a navigation bar?: 0.41662236864106983
	10: TinyMCE SCRIPT5: Access is denied. Typical cross-domain error on same domain: 0.4152998579250148
	11: php sprintf replace multiple placeholders with one value: 0.4059620871040117
	12: mod_rewrite for redirecting a directory to a top 

## Save Embeddings in JSONL

- I.e. each embedding dictionary written as an individual JSON object on its own line.
- Write embeddings in batches to prevent out-of-memory errors.

This will take a few minutes.

In [20]:
import tempfile
from pathlib import Path

# Create temporary file to write embeddings to
embeddings_file_path = Path(tempfile.mkdtemp())

print(f"Embeddings directory: {embeddings_file_path}")

Embeddings directory: /var/tmp/tmp975fktgy


In [21]:
import gc
import json

# Write embeddings for 5000 questions, split into 1000 question chunks.
BQ_NUM_ROWS = 5000
BQ_CHUNK_SIZE = 1000
BQ_NUM_CHUNKS = math.ceil(BQ_NUM_ROWS / BQ_CHUNK_SIZE)

START_CHUNK = 0

# Create a rate limit of 300 requests per minute. Adjust this depending on your quota.
API_CALLS_PER_SECOND = 300 / 60
# According to the docs, each request can process 5 instances per request
ITEMS_PER_REQUEST = 5

# Loop through each generated dataframe, convert
for i, df in tqdm(
    enumerate(
        query_bigquery_chunks(
            max_rows=BQ_NUM_ROWS, rows_per_chunk=BQ_CHUNK_SIZE, start_chunk=START_CHUNK
        )
    ),
    total=BQ_NUM_CHUNKS - START_CHUNK,
    position=-1,
    desc="Chunk of rows from BigQuery",
):
    # Create a unique output file for each chunk
    chunk_path = embeddings_file_path.joinpath(
        f"{embeddings_file_path.stem}_{i+START_CHUNK}.json"
    )
    with open(chunk_path, "a") as f:
        id_chunk = df.id

        # Convert batch to embeddings
        is_successful, question_chunk_embeddings = encode_text_to_embedding_batched(
            sentences=df.title_with_body.to_list(),
            api_calls_per_second=API_CALLS_PER_SECOND,
            batch_size=ITEMS_PER_REQUEST,
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk[is_successful], question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

        # Delete the DataFrame and any other large data structures
        del df
        gc.collect()

Chunk of rows from BigQuery:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

## Upload Embeddings to GCS

In [23]:
BUCKET_URI = f"gs://{PROJECT_ID}-unique"
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

# Upload training data to a GCS bucket
remote_folder = f"{BUCKET_URI}/{embeddings_file_path.stem}/"
! gsutil -m cp -r {embeddings_file_path}/* {remote_folder}

Creating gs://qwiklabs-gcp-00-776bfcd5598f-unique/...
Copying file:///var/tmp/tmp975fktgy/tmp975fktgy_0.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp975fktgy/tmp975fktgy_1.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp975fktgy/tmp975fktgy_2.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp975fktgy/tmp975fktgy_3.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp975fktgy/tmp975fktgy_4.json [Content-Type=application/json]...
/ [5/5 files][  6.4 MiB/  6.4 MiB] 100% Done                                    
Operation completed over 5 objects/6.4 MiB.                                      


## Create an Index in Vertex AI Vector Search for the Uploading Embeddings

Create the index by reading the embeddings from the GCS bucket. This can take a long time. Maybe an hour!

In [25]:
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "question titles and bodies from stackoverflow"

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

DIMENSIONS = 768

tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=remote_folder,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=80,
    description=DESCRIPTION,
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/279850850042/locations/us-east4/indexes/3687612465979326464/operations/2162634368474939392
MatchingEngineIndex created. Resource name: projects/279850850042/locations/us-east4/indexes/3687612465979326464
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/279850850042/locations/us-east4/indexes/3687612465979326464')


In [32]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

'projects/279850850042/locations/us-east4/indexes/3687612465979326464'

In [33]:
# Retrieve an existing index by name
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

## Create and Deploy an IndexEndpoint to that it can be accesssed by API

Creating the IndexEndpoint is quick, but deploying can take about 15 minutes.

In [34]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    public_endpoint_enabled=True,
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/279850850042/locations/us-east4/indexEndpoints/5762962653639081984/operations/4585570968000266240
MatchingEngineIndexEndpoint created. Resource name: projects/279850850042/locations/us-east4/indexEndpoints/5762962653639081984
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/279850850042/locations/us-east4/indexEndpoints/5762962653639081984')


In [ ]:
# Deploy the index to the created endpoint
DEPLOYED_INDEX_ID = "deployed_index_id_unique"

DEPLOYED_INDEX_ID


my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/279850850042/locations/us-east4/indexEndpoints/5762962653639081984
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/279850850042/locations/us-east4/indexEndpoints/5762962653639081984/operations/4184750601164292096


In [ ]:
number_of_vectors = sum(
    aiplatform.MatchingEngineIndex(
        deployed_index.index
    )._gca_resource.index_stats.vectors_count
    for deployed_index in my_index_endpoint.deployed_indexes
)

print(f"Expected: {BQ_NUM_ROWS}, Actual: {number_of_vectors}")

## Query Against the Deployed Index to Find Nearest Neighbours

In [ ]:
# Create embedding for test question
test_embeddings = encode_texts_to_embeddings(sentences=["Install GPU for Tensorflow"])

In [ ]:
# Test the query to retrieve similar embeddings
NUM_NEIGHBOURS = 10

response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=NUM_NEIGHBOURS,
)

response

In [ ]:
# Check the retrieved results are relevant by actually looking at the Stack Overflow links
for match_index, neighbor in enumerate(response[0]):
    print(f"https://stackoverflow.com/questions/{neighbor.id}")

## Environment Cleanup

In [ ]:
import os

delete_bucket = False

# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

# Delete indexes
tree_ah_index.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}